# Legal Case Simulator

# Get the data from the Legal Stories GitHub repository:

In [ ]:
# Install Dependencies
import os
!pip install -q --upgrade torch
!pip install -q transformers triton==3.4 kernels
!pip uninstall -q torchvision torchaudio -y
os.environ["HF_HOME"] = "/content/drive/MyDrive/hf_cache"
os.makedirs("/content/drive/MyDrive/hf_cache", exist_ok=True)


# # Get files
!wget https://raw.githubusercontent.com/hjian42/LegalStories/main/data/101-doctrines/legal_doctrines_101.tsv
!wget https://raw.githubusercontent.com/hjian42/LegalStories/main/data/101-doctrines/gpt3.5_story_question_101.tsv

In [ ]:
# Legal Doctrines 101 file
import csv
def load_doctrines():
  # Format: concept	intro_text	word_count
  with open('legal_doctrines_101.tsv', newline='') as f:
    reader = csv.DictReader(f, delimiter='\t')
    data = list(reader)
  return data

# GPT 3.5 Story Questions 101 file
def load_story_questions():
  # Format: concept	intro_text	story	concept_question	ending_question	limitation_question
  with open('gpt3.5_story_question_101.tsv', newline='') as f:
    reader = csv.DictReader(f, delimiter='\t')
    data = list(reader)
  return data

doctrines_data = load_doctrines()
questions_data = load_story_questions()

print("Doctrines Data: ")
for row in doctrines_data:
  print(row)

print("\nStory Questions Data: ")
for row in questions_data:
  print(row)


# Prepare Data for Finetuning

In [ ]:
import sys
import os
import json

def get_question_and_concept(question):
  prompt =""
  completion = ""
  prompt += "Concept: "
  prompt += question["concept"]
  prompt += ". Summary Text: "
  prompt += question["intro_text"]

  completion += "Story: "
  completion += question["story"]
  # May not need concept question for what we want to do, can be added if needed
  # completion += " Concept Question: "
  # completion += question["concept_question"]
  # completion += " Limitation Question: "
  # completion += question["limitation_question"]
  completion += " Question: "
  completion += question["ending_question"]
  completion_new = completion.replace("\n\n", "\n")
  return prompt, completion_new


def prepare_questions_finetuning_data(raw_data, filename='finetuning_data_questions.jsonl'):
  finetuning_data = []
  for question in raw_data:
    data = {}
    prompt, completion = get_question_and_concept(question)
    data["messages"] = [{"role":"user", "content":prompt}, {"role":"assistant", "content":completion}]
    finetuning_data.append(data)
  with open(filename, 'w') as out:
    for data in finetuning_data:
      out.write(json.dumps(data))
      out.write('\n')

prepare_questions_finetuning_data(questions_data)


In [ ]:
# Print head of file
!head finetuning_data_questions.jsonl

# Finetune GPT3.5

In [ ]:
# @title
import os
import openai

print("Enter openai api key: ")
openai.api_key = input()

os.environ['OPENAI_API_KEY']=openai.api_key

In [ ]:
client = openai.OpenAI()

client.files.create(
    file=open("finetuning_data_questions.jsonl",'rb'),
    purpose="fine-tune"
)

In [ ]:
print('Training File ID:')
training_file = input()

In [ ]:
client.fine_tuning.jobs.create(
  training_file=training_file,
  model="gpt-3.5-turbo-0125"
)

In [ ]:
# Choose a random topic for the case
import random
topic = random.choice(doctrines_data)

# Create examples for the prompt
ex1in, ex1out = get_question_and_concept(questions_data[0])
ex2in, ex2out = get_question_and_concept(questions_data[1])
ex3in, ex3out = get_question_and_concept(questions_data[2])
story_so_far = None # Used to pass the previous question to prompt

# Create the prompt
# Used Chatgpt to improve the prompt to make the story continuous
def initial_prompt():
  return [
    {
        "role": "system",
        "content": (
            "You are a legal case question generator that creates a multi-step case based on legal concepts. "
            "You will produce a multiple-choice question built around a short narrative introduction. "
            "The story must introduce characters, a location, and a situation that can be expanded later."
            "You MUST write in this structure:\n\n"
            "1. STORY: A 3–6 sentence narrative introducing the situation.\n"
            "2. QUESTION: A single legal question about the concept.\n"
            "3. /ANSWER: The correct choice.\n"
            "4. RATIONALE: A short explanation.\n\n"
            "Do NOT end the story — leave open threads so the next question can continue it.\n\n"
            "Examples:\n"
            f"Concept: {ex1in}\nQuestion: {ex1out}\n"
            f"Concept: {ex2in}\nQuestion: {ex2out}\n"
            f"Concept: {ex3in}\nQuestion: {ex3out}\n"
        )
    },
    {
        "role": "user",
        "content": f"Concept: {topic['concept']}. Summary Text: {topic['intro_text']}"
    }
]

def middle_prompt(story_so_far):
  return [
    {
        "role": "system",
        "content": (
            "You are continuing a legal case story. Your job is to write the NEXT question in the same narrative. "
            "You MUST continue the same characters, same setting, same timeline, and the same story threads. "
            "Use the previous story as canon and extend it naturally.\n\n"
            "Your structure MUST be:\n"
            "1. STORY: 3–6 new sentences that continue the narrative.\n"
            "2. QUESTION: A new multiple-choice question about a NEW legal concept.\n"
            "3. /ANSWER.\n"
            "4. RATIONALE.\n\n"
            "Do not contradict earlier facts. Do not end the story — keep it open.\n\n"
            "Examples:\n"
            f"Concept: {ex1in}\nQuestion: {ex1out}\n"
            f"Concept: {ex2in}\nQuestion: {ex2out}\n"
            f"Concept: {ex3in}\nQuestion: {ex3out}\n"
        )
    },
    {
        "role": "user",
        "content": f"Story so far:\n{story_so_far}"
    }
]

def end_prompt(story_so_far):
  return [
    {
        "role": "system",
        "content": (
            "Write the final question in this legal case. Continue the story from the previous section, "
            "but this time bring the situation to a conclusion.\n\n"
            "Structure:\n"
            "1. STORY: 3–6 sentences resolving the conflict.\n"
            "2. QUESTION: A final legal concept question.\n"
            "3. /ANSWER.\n"
            "4. RATIONALE.\n\n"
            "The story should conclude all major plot threads.\n\n"
        )
    },
    {
        "role": "user",
        "content": f"Story so far:\n{story_so_far}"
    }
]

def generate_response(prompt, finetuned_model):
  response = client.chat.completions.create(
    model=finetuned_model,
    messages=prompt,
    temperature=0.7,
    max_tokens=500,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["###"]
    )
  return response.choices[0].message.content.strip()

question = generate_response(initial_prompt(), "gpt-3.5-turbo-0125")
print("BEGINNING: ")
print(question)
print("\n\n")
story_so_far = question.split("/")[0]
question2 = generate_response(middle_prompt(story_so_far), "gpt-3.5-turbo-0125")
print("MIDDLE: ")
print(question2)
print("\n\n")
story_so_far += question2.split("/")[0]
question3 = generate_response(end_prompt(story_so_far), "gpt-3.5-turbo-0125")
print("END: ")
print(question3)

# Setup Reasoning Model

In [ ]:
print("What is your response? ")
user_response = input()

In [ ]:
# Using o3-mini, GptOss was way too big for colab to handle
from openai import OpenAI
client2 = OpenAI()

o3mini_prompt = (
    "You are a legal evaluator. Given a question, the correct response, and the users response"
    "return a score out of 4 points. "
    "1 point for the correct letter answer."
    "1 point for mentioning the right doctrines/concept/keywords."
    "1 point for an okay justification or 2 points for a good justification."
    "Provide your resoning for the score. An example of how it should be formatted is below: "
    "Example: "
    "Score: 4\n"
    "Rationale: You provided the correct answer to the question, and mentioned the concept of legality."
    "Your justification could use some improvements though, it seemed somewhat ambiguous."
)

def evaluate_response(question, user_response, prompt, model):
  response = client2.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": f"Question: {question}\nUser Response: {user_response}"}
    ]
  )
  return response.choices[0].message.content

evaluate_response(question, user_response, o3mini_prompt, "o3-mini")

# Get Users answer

In [ ]:
questions_to_win = 6
story_so_far = None
user_response = ""
win = False
lose = False
count = 0

# Run game (use -1 to quit for now)
while not win and not lose and user_response != "-1":
  if count == 0:
    # Start case
    print("\nJUDGE: ")
    question = generate_response(initial_prompt(), "gpt-3.5-turbo-0125")
    print(question)
    story_so_far = question.split("/")[0]
    print("\nATTORNEY (Enter response): ")
    user_response = input()
    print("\nJURY: ")
    print(evaluate_response(question, user_response, o3mini_prompt, "o3-mini"))
  elif count == questions_to_win:
    # End case
    print("\nJUDGE: ")
    question = generate_response(end_prompt(story_so_far), "gpt-3.5-turbo-0125")
    print(question)
    print("\nATTORNEY (Enter response): ")
    user_response = input()
    print("\nJURY: ")
    print(evaluate_response(question, user_response, o3mini_prompt, "o3-mini"))
  else:
    # Middle case
    print("\nJUDGE: ")
    question = generate_response(middle_prompt(story_so_far), "gpt-3.5-turbo-0125")
    print(question)
    story_so_far += question.split("/")[0]
    print("\nATTORNEY (Enter response): ")
    user_response = input()
    print("\nJURY: ")
    print(evaluate_response(question, user_response, o3mini_prompt, "o3-mini"))
print("CASE CLOSED.")

